<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import spacy
import wikipedia
import pandas as pd
from spacy import displacy

import urllib 
import time
import requests 
from bs4 import BeautifulSoup

In [2]:
title_file = '../01_make_matching_list/page_titles.txt'

df_page_title = pd.read_csv(title_file, delimiter='\t',header=None)

In [3]:
DICT_PAGE_TITLE = dict(zip(df_page_title[0],df_page_title[1]))

In [4]:
def find_wiki_title(term):
    title = wikipedia.search(term)
    if title != []:
        return title[0]

def text_to_wiki(text, nlp): # gives the html under BeatifulSoup format
    doc = nlp(text)

    html = displacy.render(doc, style="ent", options={"ents": ["TERM"]}, jupyter=False, page=True)
    soup = BeautifulSoup(html)
    marks =  soup.find_all('mark')
    
    for mark in marks :
        try:
            entity = mark.get_text(strip=True,separator=', ').split(', ')[0] # get the entity annotated
            url = f'https://en.wikipedia.org/wiki/{"_".join(DICT_PAGE_TITLE[entity].split())}' 
        except KeyError:
            wiki_title = find_wiki_title(entity)   
            if wiki_title:
                url = f'https://en.wikipedia.org/wiki/{"_".join(wiki_title.split())}' 
                DICT_PAGE_TITLE.update({entity:wiki_title})
                    
        link = soup.new_tag('a', href=url) # create the html tag for link       
        mark.wrap(link) #add html tag <a> (the one to make links) to around our annotated word
    return soup

In [5]:
nlp = spacy.load(r"./output/G06F_2019/model-best/") #load the best model

with open('../G06T0007254000.txt') as f:
    patents = f.read()


doc = nlp(patents) # input sample text

In [6]:
open('./demo.html', 'w', encoding="utf-8").write(str(text_to_wiki(patents,nlp))) #create file with html data

809335

In [15]:
try:
    print(wikipedia.summary('SVM'))
except wikipedia.exceptions.DisambiguationError:
    print("disambiguation")

disambiguation


/Users/zuoyou/opt/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/zuoyou/opt/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
